In [2]:
import numpy as np
import pandas as pd
import sklearn
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics.pairwise import cosine_similarity
import scipy
from numba import njit
from numba import jit
from numba import vectorize
from timeit import default_timer as timer
from datetime import timedelta

In [3]:
movies = pd.read_csv('movie_archive\movies_metadata.csv')
ratings_small = pd.read_csv('movie_archive\\ratings_small.csv')
links_small = pd.read_csv('movie_archive\links_small.csv')

C:\Users\Admin\AppData\Roaming\Python\Python38\site-packages\IPython\core\interactiveshell.py:3146: DtypeWarning: Columns (10) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


In [4]:
%%time
tfidf = CountVectorizer(analyzer='word', ngram_range=(1, 3), stop_words='english')
tfidf_matrix = tfidf.fit_transform(movies['title'].apply(str) + " " + movies['overview'].apply(str))
tfidf_matrix

Wall time: 14.1 s


<45466x2407186 sparse matrix of type '<class 'numpy.int64'>'
	with 3943994 stored elements in Compressed Sparse Row format>

In [5]:
userid_list = ratings_small['userId'].unique()
movieid_list = movies['id'].to_list()
# changes movieId type to int and deletes all false entrys
movieid_list = [x for x in movieid_list if x.isdigit()]
movieid_list = list(map(int, movieid_list))
movieid_list = np.array(movieid_list)

# get all movie vector rated from user
def get_user_rated_movies(tmdbs):
    movie_vector_list = []
    for i,id in enumerate(tmdbs):
        index = np.where(movieid_list == id)
        movie_vector = tfidf_matrix[index]
        movie_vector_list.append(scipy.sparse.csr_matrix.toarray(movie_vector)) 
        
    movie_vectors = np.array(movie_vector_list)
    #movie_vectors = scipy.sparse.vstack(movie_vector_list)

    return movie_vectors


# get rated movies from user, convert mid to tmdbid
def get_user_profile(uid):
    user_profile = ratings_small.loc[ratings_small['userId'] == uid]
    user_movieid_list = user_profile['movieId'].to_list()
    user_movieid_tmdbId = np.zeros_like(user_movieid_list)
    for i, mid in enumerate(user_movieid_list):
        tmdbId = links_small['tmdbId'].loc[links_small['movieId'] == uid]
        try:
            tmdbId = int(tmdbId)
            user_movieid_tmdbId[i] = tmdbId
        except ValueError, TypeError:
            print ("ValueError tmdbId: " + str(tmdbId))

    return user_movieid_tmdbId


@njit(fastmath=True, parallel=True)
def create_user_vector(user_rated_movies_vector_array):
    user_vector = user_rated_movies_vector_array[0]

    for x in user_rated_movies_vector_array[1:]:
        user_vector = user_vector + x
    
    return user_vector    
        

# compute all movie vector form specific user into one vector for each user
all_user_profiles = {}
def create_user_vectors():
    for uid in userid_list:
        user_tmdbid_array = get_user_profile(uid)
        
        n = len(user_tmdbid_array)
        if n < 1000:
            user_rated_movies_vector_array = get_user_rated_movies(user_tmdbid_array)

            print("Time Compute User Vector: n = " + str(n))
            start = timer()
            user_vector = create_user_vector(user_rated_movies_vector_array)
            end = timer()
            print ("Execution time HH:MM:SS: ",timedelta(seconds=end-start))

            user_norm = sklearn.preprocessing.normalize(user_vector)
            all_user_profiles[uid] = user_norm
            print(str(uid) + " finished\n")
        else:
            print("Skipped " + str(uid))
            print("Samples: " + str(n) +"\n")
        

%time create_user_vectors()

Time Compute User Vector: n = 20
Execution time HH:MM:SS:  0:00:00.827903
1 finished

Time Compute User Vector: n = 76
Execution time HH:MM:SS:  0:00:00.543062
2 finished

Time Compute User Vector: n = 51
Execution time HH:MM:SS:  0:00:00.374286
3 finished

Time Compute User Vector: n = 204
Execution time HH:MM:SS:  0:00:01.633062
4 finished

Time Compute User Vector: n = 100
Execution time HH:MM:SS:  0:00:00.720478
5 finished

Time Compute User Vector: n = 44
Execution time HH:MM:SS:  0:00:00.327825
6 finished

Time Compute User Vector: n = 88
Execution time HH:MM:SS:  0:00:00.637999
7 finished

Time Compute User Vector: n = 116
Execution time HH:MM:SS:  0:00:00.829814
8 finished

Time Compute User Vector: n = 45
Execution time HH:MM:SS:  0:00:00.328492
9 finished

Time Compute User Vector: n = 46
Execution time HH:MM:SS:  0:00:00.331603
10 finished

Time Compute User Vector: n = 38
Execution time HH:MM:SS:  0:00:00.277631
11 finished

Time Compute User Vector: n = 61
Execution time H

TypeError: cannot convert the series to <class 'int'>

Time Compute User Vector: n = 20
Execution time HH:MM:SS:  0:00:00.152222
1 finished

Time Compute User Vector: n = 76
Execution time HH:MM:SS:  0:00:00.610072
2 finished

Time Compute User Vector: n = 51
Execution time HH:MM:SS:  0:00:00.428506
3 finished

Time Compute User Vector: n = 204
Execution time HH:MM:SS:  0:00:01.827316
4 finished

Time Compute User Vector: n = 100
Execution time HH:MM:SS:  0:00:00.864352
5 finished

Time Compute User Vector: n = 44
Execution time HH:MM:SS:  0:00:00.351326
6 finished

Time Compute User Vector: n = 88
Execution time HH:MM:SS:  0:00:00.765186
7 finished

Time Compute User Vector: n = 116
Execution time HH:MM:SS:  0:00:01.097722
8 finished

Time Compute User Vector: n = 45
Execution time HH:MM:SS:  0:00:00.389946
9 finished

Time Compute User Vector: n = 46
Execution time HH:MM:SS:  0:00:00.421764
10 finished

Time Compute User Vector: n = 38
Execution time HH:MM:SS:  0:00:00.318130
11 finished



Time Compute User Vector: n = 20
Execution time HH:MM:SS:  0:00:01.222243
1 finished

Time Compute User Vector: n = 76
Execution time HH:MM:SS:  0:00:00.557548
2 finished

Time Compute User Vector: n = 51
Execution time HH:MM:SS:  0:00:00.370323
3 finished

Time Compute User Vector: n = 204
Execution time HH:MM:SS:  0:00:01.454427
4 finished

Time Compute User Vector: n = 100
Execution time HH:MM:SS:  0:00:00.705948
5 finished

Time Compute User Vector: n = 44
Execution time HH:MM:SS:  0:00:00.319584
6 finished

Time Compute User Vector: n = 88
Execution time HH:MM:SS:  0:00:00.629733
7 finished

Time Compute User Vector: n = 116
Execution time HH:MM:SS:  0:00:00.867882
8 finished

Time Compute User Vector: n = 45
Execution time HH:MM:SS:  0:00:00.333555
9 finished

Time Compute User Vector: n = 46
Execution time HH:MM:SS:  0:00:00.333807
10 finished

Time Compute User Vector: n = 38
Execution time HH:MM:SS:  0:00:00.260933
11 finished